In [1]:
import numpy as np

In [3]:
# Sets (times, locations)

class Time:

    def __init__(self, time, order, prep):

        self.time = time # Name of time period
        self.order = order # Order of time period in the day
        self.prep = prep # Preposition, used for natural language processing when referring to time

    def __repr__(self):
        return str(self.time)

    def __lt__(self, other):
        return self.order < other.order

    def __gt__(self, other):
        return self.order > other.order

class Location:

    def __init__(self, location, prep):

        self.location = location # Name of location
        self.prep = prep # Preposition, used for natural language processing when referring to location
        self.clues = []

    def __repr__(self):
        return str(self.location)

    def get_present_characters(self, time, characters):
        # Returns list of characters present at location at given time
        present_characters = []
        for character in characters:
            if character.timeline[time] == self:
                present_characters.append(character)
        return present_characters

# Characters

class Character:

    def __init__(self, name, data):
        self.name = name
        self.data = data
        self.clues = []

    def __repr__(self):
        return self.name

    def generate_timeline(self, times, locations):
        self.timeline = {time: np.random.choice(locations) for time in times}
    
    def get_observed_characters(self, time, characters):
        # Returns list of other characters observed by character at given time
        observed_characters = []
        for character in characters:
            if (character != self) and (self.timeline[time] == character.timeline[time]):
                observed_characters.append(character)
        return observed_characters

# Clues

class Clue:

    def __init__(self, description, relevant):
        
        self.description = description
        self.relevant = relevant

class PhysicalClue(Clue):

    def __init__(self, location, **kwargs):
        
        super().__init__(**kwargs)
        
        # Define attributes of physical clue
        self.physical = True
        self.location = location

class VerbalClue(Clue):
    
    def __init__(self, character, **kwargs):
        
        super().__init__(**kwargs)
        
        # Define attributes of verbal clue
        self.verbal = True
        self.character = character
        

# Murder Scenario

class MurderScenario:

    def __init__(self, murderer, victim, time, location):
        self.murderer = murderer
        self.victim = victim
        self.time = time
        self.location = location

    def __repr__(self):
        return f"{self.murderer} murders {self.victim} {self.time.prep} {self.time.time.lower()} at {self.location.prep} {self.location.location.lower()}"

    def activate(self, game):
        # Activate murder scenario
        self.adjust_timelines(game)
        
        # Generate alibi for murderer
        self.generate_alibi(game)

        # Generate motive for murderer
        self.generate_motive(game)

        # Generate other clues and evidence
        self.generate_clues(game)

    def adjust_timelines(self, game):
        
        # Ensure murderer and victim at the same location at time of the murder
        self.murderer.timeline[self.time] = self.location
        self.victim.timeline[self.time] = self.location

        # Adjust victim's timeline after the murder
        for time in game.times:
            if time > self.time:
                self.victim.timeline[time] = Location("Unknown", "")

    def generate_alibi(self, game):

        # Note: characters will only give their alibi to the detective, NOT their true timeline
        # For most characters, their alibi is their true timeline
        # For the murderer, their alibi is their true timeline except for the time of the murder

        # Create alibis of all characters
        for character in game.characters:
            character.alibi = character.timeline.copy()
            
            # Fix each character's observations of other characters
            character.observed_characters = {}
            for time in game.times:
                character.observed_characters[time] = character.get_observed_characters(time, game.characters)
        
        # Get list of locations with no or few other characters present at the time of the murder
        candidate_locations = []
        k = 0 # Number of other characters present at alibi location
        while candidate_locations == []:
            for location in game.locations:
                # Alibi location cannot be location of murder
                if location != self.location:
                    if len(location.get_present_characters(self.time, game.characters)) == k:
                        candidate_locations.append(location)
            k += 1
        k -= 1
        
        # Randomly select alibi location
        alibi_location = np.random.choice(candidate_locations)

        # Update murderer alibi
        self.murderer.alibi[self.time] = alibi_location
        self.murderer.observed_characters[self.time] = list(
            np.random.choice(
                [character for character in characters if not (character == self.murderer or character == self.victim)],
                k,
            )
        )

    def generate_motive(self, game):
        pass
    
    def generate_clues(self, game):
        pass


# Game

class Game:

    def __init__(self, times, locations, characters):

        self.times = times
        self.locations = locations
        self.characters = characters

        # Initialize character timelines
        for character in self.characters:
            character.generate_timeline(self.times, self.locations)

        # Initialize list of collected clues
        self.clues = []

    def generate_scenario(self):
        # List of candidate murder scenarios
        candidate_scenarios = []
        # Loop through all possible combinations of murderer / victim
        for murderer in self.characters:
            for victim in self.characters:
                # Murderer cannot be victim
                if murderer != victim:
                    # Loop through all possible times
                    for time in self.times:
                        # If murderer and victim alone at the same location at the same time, or alone at different locations at the same time
                        if (
                            murderer.get_observed_characters(time, self.characters) == [victim] 
                            or (murderer.get_observed_characters(time, self.characters) == [] and victim.get_observed_characters(time, self.characters) == [])
                           ):
                            location = victim.timeline[time]
                            candidate_scenarios.append(MurderScenario(murderer, victim, time, location))
                            
        # Randomly choose scenario and activate (adjust timelines, create alibi, evidence)
        self.scenario = np.random.choice(candidate_scenarios)
        self.scenario.activate(self)

    def interview_character(self, character):
        pass

    def inspect_location(self, location):
        pass
        

In [ ]:
def start_game():
    pass

def 

In [15]:
# List of times, locations, and characters as data
time_kwargs = [("Morning", 0, "in the"), ("Afternoon", 1, "in the"), ("Night", 2, "at")]
location_kwargs = [("Harbor", "the"), ("Beach", "the"), ("Tavern", "the"), ("Lighthouse", "the")]
character_kwargs = [("Alice", ""), ("Bob", ""), ("Charlie", ""), ("Victor", ""), ("James", ""), ("Margaret", "")]

# Get list of times, locations, and characters as objects
times = [Time(*kwargs) for kwargs in time_kwargs]
locations = [Location(*kwargs) for kwargs in location_kwargs]
characters = [Character(*kwargs) for kwargs in character_kwargs]

# Initialize game
game = Game(times, locations, characters)

# Select murder scenario
game.generate_scenario()

print(game.scenario)
print("\n")

for character in characters:
    print(character)
    print(" | ".join([f"{time}: {character.alibi[time]} --> {character.observed_characters[time]}" for time in game.times]) + "\n")

Victor murders Charlie in the morning at the tavern


Alice
Morning: Lighthouse --> [Bob] | Afternoon: Tavern --> [] | Night: Lighthouse --> [Victor, James, Margaret]

Bob
Morning: Lighthouse --> [Alice] | Afternoon: Harbor --> [Victor, James] | Night: Beach --> []

Charlie
Morning: Tavern --> [Victor] | Afternoon: Unknown --> [] | Night: Unknown --> []

Victor
Morning: Beach --> [] | Afternoon: Harbor --> [Bob, James] | Night: Lighthouse --> [Alice, James, Margaret]

James
Morning: Harbor --> [Margaret] | Afternoon: Harbor --> [Bob, Victor] | Night: Lighthouse --> [Alice, Victor, Margaret]

Margaret
Morning: Harbor --> [James] | Afternoon: Beach --> [] | Night: Lighthouse --> [Alice, Victor, James]



In [16]:
game

In [17]:
game.interview_character()

TypeError: Game.interview_character() missing 1 required positional argument: 'character'

### Outcomes of Discussion with Aaron

1. Depending on the location of the murder, the body is randomly moved to a certain location (cliffs, beach, harbor, etc.). and a trail of physical evidence is left (a weapon, belongings, bloody footprint/handprint, etc.) along with the body.
2. Part of the gameplay involves finding the body, determining cause and location of death, and piecing together timeline of the murder to rule out certain characters. Furthermore, the gameplay involves narrowing down the list of suspects using information about their motives.
3. Game generates a network of motives (based on archetypes) and randomly imposes them on the cast of characters; these motives can either be procedurally generated eventually, but initially we will fix the relative network of motives and randomly impose them on the characters. The detective must discover the network of motives to narrow down the list of suspects.
4. Characters may randomly reveal their knowledge of their own and other characters' motives to the detective. Physical evidence of motives may also exist in the suspects or victim's belongings. The motive network is what may create red herrings.
5. The detective must work discreetly to avoid alerting the suspects and the murderer. If he asks questions that are too forthright to the suspects involved in the case, the murderer may become hostile and retaliate. The game will alert the detective that the murderer is wary of him, and may even send the detective a hostile warning message. The detective effectively has a "budget" of interview questions and investigations which he may conduct, hence he must use his intuition to look for clues in the right places. If he uses his budget up or too much is revealed without the detective confronting the murderer, the murderer may confront the detective and kill him. The challenge of the game is to use intuition and deduction to efficiently look for clues and solve the murder.

In [14]:
import pandas as pd
import pathlib

curr_dir = pathlib.Path(".")

df = pd.read_csv(curr_dir / "Data" / "Character_Location_Probabilities" / "Alice_Location_Probs.csv", index_col=[0])

df

,Morning,Afternoon,Night
Harbor,0.1,0.1,0.05
Beach,0.1,0.5,0.05
Tavern,0.5,0.3,0.05
Lighthouse,0.3,0.1,0.85
